# KoBERT

In [1]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# KoBERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

file_path = 'data/'

df = pd.read_excel(f'{file_path}law_recomended_0712.xlsx')

# 판례 데이터와 벡터화
case_texts = df['court_detail'].tolist()
embeddings = np.vstack([get_embedding(text) for text in tqdm(case_texts)])

# 새로운 상황 텍스트와 벡터화
new_case_text = "새로운 상황 텍스트"
new_embedding = get_embedding(new_case_text)

# 유사도 계산
similarities = cosine_similarity(new_embedding, embeddings)

# 상위 5개 판례 선택
top_5_indices = similarities[0].argsort()[-5:][::-1]
top_5_cases = [case_texts[i] for i in top_5_indices]
print(top_5_cases)


C:\Users\reals\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\reals\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\reals\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


  0%|          | 0/87444 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
import pandas as pd

df = pd.read_excel('data/law_recomended.xlsx')
df


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\reals\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\reals\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\reals\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\reals\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "c:\Users\reals\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\reals\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\reals\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\reals\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\Users\reals\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "c:\Users\reals\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

FileNotFoundError: [Errno 2] No such file or directory: 'data/law_recomended.xlsx'

In [5]:
!pip install --upgrade pip #sklearn huggingface_hub transformers datasets langchain konlpy

ERROR: Invalid requirement: 'pip#sklearn': Expected end or semicolon (after name and no valid version specifier)
    pip#sklearn
       ^

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install --upgrade pip

   ---------------------------------------- 1.8/1.8 MB 13.0 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\reals\anaconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
embeddings = embed_text_in_batches(df['prepro_detail'].tolist())

# KMeans 군집화
n_clusters = 7  # 원하는 군집 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from konlpy.tag import Komoran
import pandas as pd
import torch
import re
import os

# 텍스트 전처리
def prepro_text(text) :
    #text = re.sub(r'\w+', ' ', text)
    tokens = komoran.morphs(text)
    # 토큰을 공백으로 연결
    processed_text = ' '.join(tokens)
    return text

# 텍스트 임베딩 함수 (배치 처리)
def embed_text_in_batches(text_list, batch_size=16):
    embeddings = []
    for i in range(0, len(text_list), batch_size):
        batch_texts = text_list[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=256)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings)

# colab 사용 시 /content/ 내 작업공간 사용 시 data/
file_path = 'data/'
model_name = "beomi/KcELECTRA-base-v2022"

# 데이터, 모델, 토크나이저 로드
f = open(f'{file_path}hugging_face_token_key.txt', 'r')
my_key = f.readline()

#os.environ['HUGGINGFACEHUB_API_TOKEN'] = my_key

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

df = pd.read_excel(f'{file_path}law_recomended_0712.xlsx').dropna(axis=0)
df = df[:20000]
# 전처리된 column 가져오기
df['detail_mean'] = df['court_detail'].str.extract(r'【이    유】(.*)', expand=False)

# 실행
komoran = Komoran()
#df = df[:20]

df['detail_mean'] = df['detail_mean'].str.replace('\n', '')
df['detail_mean'] = df['detail_mean'].fillna('')
#df['prepro_detail'] = df['detail_mean'].apply(prepro_text)
df['prepro_detail'] = df['detail_mean'].apply(prepro_text)

embeddings = embed_text_in_batches(df['prepro_detail'].tolist())

# KMeans 군집화
n_clusters = 7  # 원하는 군집 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)



C:\Users\reals\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\reals\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\reals\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


NameError: name 'embed_text' is not defined

In [9]:
# 텍스트 임베딩 함수 (배치 처리)
def embed_text_in_batches(text_list, batch_size=1):
    embeddings = []
    for i in range(0, len(text_list), batch_size):
        batch_texts = text_list[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings)

embeddings = embed_text_in_batches(df['prepro_detail'].tolist())

# KMeans 군집화
n_clusters = 7  # 원하는 군집 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 68742807552 bytes.

In [3]:
import joblib

# 모델 및 데이터 저장
#joblib.dump(komoran, f'{file_path}komoran.pkl')
joblib.dump(tokenizer, f'{file_path}tokenizer.pkl')
torch.save(model.state_dict(), f'{file_path}/kc_electra_model.pth')
joblib.dump(embeddings, f'{file_path}embeddings.pkl')
joblib.dump(kmeans, f'{file_path}kmeans_model.pkl')
df.to_pickle(f'{file_path}data_with_preprocessed_text.pkl')

print("모델과 데이터를 파일 저장 완료")

모델과 데이터를 파일 저장 완료


# KoBERT

In [ ]:
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

In [ ]:
# 데이터 파일 불러오기
file_path = 'data/law_recomended.xlsx'
df = pd.read_excel(file_path)

# 전처리된 column 가져오기
df_detail = df['court_detail']
df_detail = df_detail.str.extract(r'【이    유】(.*)', expand=False)

# KoBERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 텍스트 데이터를 KoBERT 입력 형식으로 변환
def preprocess_text(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    return inputs

# 문서 임베딩 생성 함수
def get_embedding(text):
    inputs = preprocess_text(text)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# 모든 문서에 대해 임베딩 생성
embeddings = df_detail.apply(lambda x: get_embedding(x) if pd.notnull(x) else None)

# 결과 확인
print(embeddings.head())

In [ ]:
# 임베딩 데이터를 문자열에서 숫자 배열로 변환하는 함수
def str_to_array(s):
    return np.fromstring(s.strip('[]'), sep=' ')

# 임베딩 데이터 불러오기
loaded_embeddings_df = pd.read_excel('data/embeddings_data.xlsx')

# 문자열 데이터를 숫자 배열로 변환
loaded_embeddings = loaded_embeddings_df.iloc[:, 0].apply(lambda x: str_to_array(x) if pd.notnull(x) else np.nan)

# 임베딩 데이터를 DataFrame으로 변환 및 결측치 처리
embeddings_array = np.vstack(loaded_embeddings.dropna())

# PCA 차원 축소
pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings_array)

# 시각화
plt.scatter(pca_result[:, 0], pca_result[:, 1])
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('PCA of Document Embeddings')
plt.show()

In [ ]:
def recommend_court(input_sentence, top_n=5):
    # 입력 문장을 임베딩 변환
    query_embedding = get_embedding(input_sentence).reshape(1, -1)
    
    # 유사도 계산
    similarities = cosine_similarity(query_embedding, embeddings_array)[0]
    similar_docs_indices = np.argsort(similarities)[-top_n:][::-1]  # 상위 N개 문서 인덱스
    
    recommendations = []
    for idx in similar_docs_indices:
        if similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], 
                                df.iloc[idx]['event_num'], 
                                df.iloc[idx]['event_type'], 
                                df.iloc[idx]['jPrecedent_num'], 
                                df.iloc[idx]['court_name'], 
                                df.iloc[idx]['court_detail'], 
                                df.iloc[idx]['court_reference'], 
                                df.iloc[idx]['court_Decision'], 
                                similarities[idx]*100))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations


# 문자열 데이터를 숫자 배열로 변환
loaded_embeddings = loaded_embeddings_df.iloc[:, 0].apply(lambda x: str_to_array(x) if pd.notnull(x) else np.nan)

# 임베딩 데이터를 DataFrame으로 변환 및 결측치 처리
embeddings_array = np.vstack(loaded_embeddings.dropna())

# 원본 데이터 불러오기
df = pd.read_excel('data/law_recomended.xlsx')

# 판례 추천
query = input("상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :\n")
recommended_court = recommend_court(query)

if isinstance(recommended_court, str):
    print(recommended_court)
else:
    court_detail_OX = input("판례 및 판시 사항을 보시겠습니까? (Y/N): \n")
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_detail, court_reference, court_Decision, similarity in recommended_court:
        print('==================================================================================================================')
        if (court_detail_OX == 'N') or (court_detail_OX == 'n'):
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\n판례 일련번호 : {jPrecedent_num} \n법원 : {court_name} \n\n참조조문 \n{court_reference}\n\n\n")
        else:
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\n판례 일련번호 : {jPrecedent_num} \n법원 : {court_name} \n\n판례 내용 :\n{court_detail} \n\n참조조문 \n{court_reference} \n\n판시사항 :\n{court_Decision}\n\n\n")
        print('==================================================================================================================')


In [8]:
import torch
import joblib
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from konlpy.tag import Komoran
from sklearn.metrics.pairwise import cosine_similarity

# colab 사용 시 /content/ 내 작업공간 사용 시 data/ /content/drive/MyDrive/data/
file_path = 'data/'

# 데이터, 모델, 토크나이저 로드
tokenizer = joblib.load(f'{file_path}tokenizer.pkl')
model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")
model.load_state_dict(torch.load(f'{file_path}kc_electra_model.pth', map_location=torch.device('cpu')))
model.eval()

# Komoran 객체 생성
komoran = Komoran()

embeddings = joblib.load(f'{file_path}embeddings.pkl')
kmeans = joblib.load(f'{file_path}kmeans_model.pkl')
df = pd.read_pickle(f'{file_path}data_with_preprocessed_text.pkl')

# 텍스트 전처리 함수
def prepro_text(text):
    tokens = komoran.morphs(text)
    processed_text = ' '.join(tokens)
    return processed_text

# 텍스트 임베딩 생성 함수
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1)
    return embedding

# 판례 추천 함수
def recommend_court(query, top_k=5):
    preprocessed_query = prepro_text(query)
    query_embedding = embed_text(preprocessed_query)
    
    # 유사도 계산
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    
    # 유사도 기준으로 상위 top_k 개 판례 선택
    top_indices = similarities.argsort()[-top_k:][::-1]
    recommended_courts = df.iloc[top_indices]
    
    results = []
    for idx in top_indices:
        result = (
            df.iloc[idx]['event_Title'],
            df.iloc[idx]['event_num'],
            df.iloc[idx]['event_type'],
            df.iloc[idx]['jPrecedent_num'],
            df.iloc[idx]['court_name'],
            df.iloc[idx]['court_detail'],
            df.iloc[idx]['court_reference'],
            df.iloc[idx]['court_Decision'],
            similarities[idx] * 100  # 유사도를 퍼센트로 변환
        )
        results.append(result)
    
    return results

# 사용자 질의 입력
query = input("상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :\n")
recommended_courts = recommend_court(query)

if isinstance(recommended_courts, str):
    print(recommended_courts)
else:
    court_detail_OX = input("판례 및 판시 사항을 보시겠습니까? (Y/N): \n")
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_detail, court_reference, court_Decision, similarity in recommended_courts:
        print('==================================================================================================================')
        if (court_detail_OX == 'N') or (court_detail_OX == 'n'):
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\n판례 일련번호 : {jPrecedent_num} \n법원 : {court_name} \n\n참조조문 \n{court_reference}\n\n\n")
        else:
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\n판례 일련번호 : {jPrecedent_num} \n법원 : {court_name} \n\n판례 내용 :\n{court_detail} \n\n참조조문 \n{court_reference} \n\n판시사항 :\n{court_Decision}\n\n\n")
        print('==================================================================================================================')


C:\Users\reals\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :
술 먹고 집으로 가던 중 치마 입은 모르는 여자분의 엉덩이를 만지고 지나갔나봐요 정말 실수로 그랬다고 합니다  그 여자분은 바로 신고했는데 경찰 말이 동네 고등학생이라고 했네요
판례 및 판시 사항을 보시겠습니까? (Y/N): 
y
사건 명 : 유족급여및장의비부지급처분취소 (유사도: 91.57%)
사건 번호 : 2015구합74692 
사건 종류 : 일반행정
판례 일련번호 : 186061 
법원 : 서울행정법원 

판례 내용 :
【원    고】 【피    고】 근로복지공단【변론종결】2016. 2. 25.【주    문】  1. 원고의 청구를 기각한다.  2. 소송비용은 원고가 부담한다.【청구취지】  피고가 2014. 12. 23. 원고에 대하여 한 유족급여 및 장의비 부지급 처분을 취소한다.【이    유】1. 처분의 경위  가. 원고의 배우자인 망 소외 1[1964. (생월일 생략)생, 이하 ‘망인’이라 한다]은 1992. 7. 1. 유리병을 제조하는 주식회사 △△(이하 ‘소외 회사’라 한다)에 입사하여 □□공장 생산팀 제병 C조 반장으로 근무하였다.  나. 망인은 2014. 7. 16. 22:00경 야간근무를 위해 소외 회사에 출근하였다가 22:30경부터 소외 회사로부터 지급받은 야식비를 회식 불참자에게 분배하여 줄 것인지 여부를 놓고 같은 조 직원인 소외 2[1972. (생월일 생략)생]와 말다툼을 시작하게 되었다.  다. 말다툼이 10분 정도 진행되었을 때 소외 2가 망인에게 “야식비를 회식 불참자에게 나누어주지 않으면 이는 엄연히 갈취나 마찬가지이다.”라는 취지의 발언을 하였고, 이에 격분한 망인이 주먹으로 소외 2의 얼굴을 때리면서 몸싸움이 시작되어 두 사람은 서로 엉겨붙은 채 바닥을 수차례 구르게 되었다.  라. 이를 발견한 동료 직원들이 두 사람을 떼어놓아 망인은 작업장으로, 소외 2는 제병공 휴게실로 이동하게 되었는데, 분을 참지 못한 망인이 몇 분 뒤 대걸레 막대기를 들고 휴게실로 

In [10]:
# 사용자 질의 입력
query = input("상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :\n")
recommended_courts = recommend_court(query)

if isinstance(recommended_courts, str):
    print(recommended_courts)
else:
    court_detail_OX = input("판례 및 판시 사항을 보시겠습니까? (Y/N): \n")
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_detail, court_reference, court_Decision, similarity in recommended_courts:
        print('==================================================================================================================')
        if (court_detail_OX == 'N') or (court_detail_OX == 'n'):
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\n판례 일련번호 : {jPrecedent_num} \n법원 : {court_name} \n\n참조조문 \n{court_reference}\n\n\n")
        else:
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\n판례 일련번호 : {jPrecedent_num} \n법원 : {court_name} \n\n판례 내용 :\n{court_detail} \n\n참조조문 \n{court_reference} \n\n판시사항 :\n{court_Decision}\n\n\n")
        print('==================================================================================================================')


상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :
이탈거래처에 대하여 유지보수작업을 마친 후 그 작업에 따라 변경된 원본 프로그램을 원고의 파일서버에 저장하지 아니하였거나, 퇴사 당시 원고에게 그들이 사용하던 원고의 영업자료와 원본 프로그램이 저장된 원고 소유의 노트북을 반납하지 아니하였다 하더라도, 원고의 웹하드에 원고의 이탈거래처의 변경된 원본 프로그램이 저장되어 있었고, 원고의 이탈거래처 서버에도 원본 프로그램 및 실행파일 등이 저장되어 있었던 것으로 보이는 이상, 그와 같은 사정만으로 피고들이 원고의 이탈거래처에 대한 유지보수작업을 방해하였다고
판례 및 판시 사항을 보시겠습니까? (Y/N): 
y
사건 명 : 건물등철거·소유권확인 (유사도: 94.27%)
사건 번호 : 2013다43666 
사건 종류 : 민사
판례 일련번호 : 172481 
법원 : 대법원 

판례 내용 :
【원고(반소피고), 피상고인】 사회복지법인 광주여자기독교청년회복지사업위원회 (소송대리인 법무법인 바른길 담당변호사 김용일 외 4인)【피고(반소원고), 상고인】 【원심판결】 광주지법 2013. 5. 15. 선고 2012나13701, 13718 판결【주    문】  원심판결을 파기하고, 사건을 광주지방법원으로 환송한다.【이    유】  상고이유에 대하여 판단한다.  1. 원심판결 이유에 의하면, 원심은 소외 1이 1966. 7. 1.경부터 이 사건 (지번 1 생략) 대지 중 판시 (다), (라)부분을 점유해 오다가 2003. 3. 21.경부터 위 부분을 피고 1이 점유해 온 사실과, 소외 2가 1966. 8. 30.경부터 이 사건 (지번 1 생략) 대지 중 판시 (나)부분을 점유하다가 소외 3, 4, 5 등을 거쳐 소외 6에게 그 부분 점유가 차례로 승계되었고 그 후 피고 2가 소외 6의 상속인으로서 그 점유를 승계받은 사실을 인정하고도, 제1심증인 소외 7의 증언 등에 의하면 1974. 10. 14.경 이 사건 (지번 1 생략) 대지에 관하여 경계측량이 이루어졌는데 

# RAG Model

In [ ]:
from datasets import load_dataset
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
import torch



In [ ]:
#!pip install huggingface_hub
#huggingface-cli login

# 데이터 전처리 후 TF-IDF, cosine similarity

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from konlpy.tag import Komoran

def preprocess_text(data):
    special_appellant = []
    original_decision = []
    order = []
    reason = []
    
    for case in data:
        try:
            special_appellant.append(re.search(r'【특별항고인】\s*(.*?)\s*【', case, re.DOTALL).group(1).strip())
        except AttributeError:
            special_appellant.append('')

        try:
            original_decision.append(re.search(r'【원심결정】\s*(.*?)\s*【', case, re.DOTALL).group(1).strip())
        except AttributeError:
            original_decision.append('')

        try:
            order.append(re.search(r'【주\s*문】\s*(.*?)\s*【', case, re.DOTALL).group(1).strip())
        except AttributeError:
            order.append('')

        try:
            reason.append(re.search(r'【이\s*유】\s*(.*)', case, re.DOTALL).group(1).strip())
        except AttributeError:
            reason.append('')

    return {
        '특별항고인': special_appellant,
        '원심결정': original_decision,
        '주문': order,
        '이유': reason
    }

# Excel 파일에서 데이터 읽기
df_law = pd.read_excel('data/law_recomended.xlsx')
df_law_detail = df_law['court_detail'].tolist()

# 판례 데이터 전처리
case_data = preprocess_text(df_law_detail)

# 판례 데이터를 데이터프레임으로 변환
df = pd.DataFrame(case_data)

# Komoran 형태소 분석기를 이용한 토큰화
km = Komoran()

# 형태소 분석을 문자열로 변환
df['reason_token'] = df['이유'].apply(lambda x: ' '.join(km.morphs(x)))

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['reason_token'])

# 특정 판례와 유사한 판례 출력
def find_similar_cases(case, top_n=5):
    # 입력된 상황을 TF-IDF로 변환
    case_tfidf = tfidf_vectorizer.transform([case])
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(case_tfidf, tfidf_matrix)
    
    # 유사도 계산 결과를 데이터프레임으로 변환
    similarity_df = pd.DataFrame(cosine_sim, columns=df.index)
    
    # 가장 유사한 판례 인덱스 찾기
    similar_cases = similarity_df.loc[0].sort_values(ascending=False).head(top_n)
    return df.loc[similar_cases.index]

# 상황 입력
case_input = str(input("상황을 입력하세요: "))

# 입력된 상황과 유사한 판례 찾기
similar_cases = find_similar_cases(case_input)

print(similar_cases)


In [ ]:
df.to_excel('data/similarty.xlsx')

In [ ]:
# 특정 판례와 유사한 판례 출력
def find_similar_cases(case, top_n=5):
    # 입력된 상황을 TF-IDF로 변환
    case_tfidf = tfidf_vectorizer.transform([case])
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(case_tfidf, tfidf_matrix)
    
    # 유사도 계산 결과를 데이터프레임으로 변환
    similarity_df = pd.DataFrame(cosine_sim, columns=df.index)
    
    # 가장 유사한 판례 인덱스 찾기
    similar_cases = similarity_df.loc[0].sort_values(ascending=False).head(top_n)
    return [df.loc[similar_cases.index], similarity_df]

# 상황 입력
case_input = str(input("상황을 입력하세요: "))

# 입력된 상황과 유사한 판례 찾기
similar_cases = find_similar_cases(case_input)

print(similar_cases)


## hugging hub pipeline

In [ ]:
#!pip install transformers
#!pip install transformers[sentencepiece]





In [ ]:
from transformers import pipeline

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration

tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

input_dict = tokenizer.prepare_seq2seq_batch("who holds the record in 100m freestyle", return_tensors="pt") 

generated = model.generate(input_ids=input_dict["input_ids"]) 
print(tokenizer.batch_decode(generated, skip_special_tokens=True)[0]) 



# DistilBERT BASE QA Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
import torch
import re

In [ ]:
!pip install textrankr
#!pip install --upgrade pip

In [ ]:
from typing import List
from konlpy.tag import Okt

class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.phrases(text)
        return tokens

    
from typing import List
from textrankr import TextRank

# 파일 경로
file_path = 'data/context.xlsx'

# 파일 로드
df = pd.read_excel(file_path)
df = df[:4]
df = df[['event_Title','court_detail']]

text = df['court_detail']

mytokenizer: OktTokenizer = OktTokenizer()
textrank: TextRank = TextRank(mytokenizer)

k: int = 3  # num sentences in the resulting summary

# summarized: str = textrank.summarize(text, k)
# print(summarized)  # gives you some text

# if verbose = False, it returns a list
summaries: List[str] = textrank.summarize(text, k, verbose=False)

In [ ]:
import pandas as pd
import numpy as np
from gensim import corpora
from gensim.models import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim_models

# 파일 경로
file_path = 'data/context.xlsx'

# 파일 로드
df = pd.read_excel(file_path)
df = df[:4]
df = df[['event_Title','court_detail']]

# 전처리 - 불용어 제거, 형태소 분석 등
texts = [[word for word in doc.split()] for doc in df['court_detail']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# LDA 모델 학습
lda_model = LdaMulticore(corpus=corpus, 
                        id2word=dictionary,
                        num_topics=10)

# 주제 추출
topics = lda_model.print_topics()
for topic in topics:
    print(topic)

# 시각화
pyLDAvis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(pyLDAvis_data, 'lda_visualization.html')

In [ ]:
# 파일 경로
file_path = 'data/law_recomended.xlsx'

# 파일 로드
df = pd.read_excel(file_path)
df = df[:4]
df = df[['event_Title','court_detail']]
df['court_detail'] = df['court_detail'].str.extract(r'【이    유】(.*)', expand=False)
df['court_detail'] = df['court_detail'].apply(lambda x: re.sub(r'[^\w\s]', '', x) if pd.notnull(x) else x)
df['court_detail'] = df['court_detail'].replace(['.', ',', '/'], '', regex=True)
df['court_detail'] = df['court_detail'].str.strip()
df.to_excel('context.xlsx')

In [ ]:
df

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering
import torch
from tqdm.notebook import tqdm

# 파일 경로
file_path = 'data/context.xlsx'

# 파일 로드
df = pd.read_excel(file_path)
df = df[:4]
df = df[['event_Title','court_detail']]

# BART 요약 모델과 토크나이저 불러오기
summarizer_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

# KoELECTRA QA 모델과 토크나이저 불러오기
qa_tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

# 각 판례의 내용을 기반으로 질문 생성
def generate_question(row):
    case_title = row.get('event_Title', '이 사건')
    case_detail = row.get('court_detail', '')

    patterns = {
        '저작권': [
            f"{case_title}에서 저작권법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 저작권법 위반 사건의 판결 결과는?",
            f"{case_title}에서 저작권 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 저작권 침해에 대한 판결은 무엇인가?",
            f"{case_title}의 저작권법 관련 판결은 무엇인가?",
        ],
        '특허': [
            f"{case_title}에서 특허법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 특허법 위반 사건의 판결 결과는?",
            f"{case_title}에서 특허 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 특허 침해에 대한 판결은 무엇인가?",
            f"{case_title}의 특허법 관련 판결은 무엇인가?",
        ],
        '상표': [
            f"{case_title}에서 상표법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 상표법 위반 사건의 판결 결과는?",
            f"{case_title}에서 상표 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 상표 침해에 대한 판결은 무엇인가?",
            f"{case_title}의 상표법 관련 판결은 무엇인가?",
        ],
        '계약': [
            f"{case_title}에서 계약법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 계약법 위반 사건의 판결 결과는?",
            f"{case_title}에서 계약 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 계약 불이행에 대한 판결은 무엇인가?",
            f"{case_title}의 계약법 관련 판결은 무엇인가?",
        ],
        '부동산': [
            f"{case_title}에서 부동산법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 부동산법 위반 사건의 판결 결과는?",
            f"{case_title}에서 부동산 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 부동산 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 부동산법 관련 판결은 무엇인가?",
        ],
        '노동': [
            f"{case_title}에서 노동법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 노동법 위반 사건의 판결 결과는?",
            f"{case_title}에서 노동 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 노동 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 노동법 관련 판결은 무엇인가?",
        ],
        '형사': [
            f"{case_title}에서 형사법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 형사법 위반 사건의 판결 결과는?",
            f"{case_title}에서 형사 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 형사 사건에 대한 판결은 무엇인가?",
            f"{case_title}의 형사법 관련 판결은 무엇인가?",
        ],
        '민사': [
            f"{case_title}에서 민사법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 민사법 위반 사건의 판결 결과는?",
            f"{case_title}에서 민사 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 민사 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 민사법 관련 판결은 무엇인가?",
        ],
        '세금': [
            f"{case_title}에서 세법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 세법 위반 사건의 판결 결과는?",
            f"{case_title}에서 세금 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 세금 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 세법 관련 판결은 무엇인가?",
        ],
        '회사': [
            f"{case_title}에서 회사법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 회사법 위반 사건의 판결 결과는?",
            f"{case_title}에서 회사 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 회사 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 회사법 관련 판결은 무엇인가?",
        ],
        '환경': [
            f"{case_title}에서 환경법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 환경법 위반 사건의 판결 결과는?",
            f"{case_title}에서 환경 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 환경 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 환경법 관련 판결은 무엇인가?",
        ],
        '교통': [
            f"{case_title}에서 교통법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 교통법 위반 사건의 판결 결과는?",
            f"{case_title}에서 교통 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 교통 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 교통법 관련 판결은 무엇인가?",
        ],
        '가족': [
            f"{case_title}에서 가족법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 가족법 위반 사건의 판결 결과는?",
            f"{case_title}에서 가족 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 가족 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 가족법 관련 판결은 무엇인가?",
        ],
        '상속': [
            f"{case_title}에서 상속법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 상속법 위반 사건의 판결 결과는?",
            f"{case_title}에서 상속 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 상속 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 상속법 관련 판결은 무엇인가?",
        ],
        '지적재산권': [
            f"{case_title}에서 지적재산권법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 지적재산권법 위반 사건의 판결 결과는?",
            f"{case_title}에서 지적재산권 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 지적재산권 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 지적재산권법 관련 판결은 무엇인가?",
        ],
        '금융': [
            f"{case_title}에서 금융법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 금융법 위반 사건의 판결 결과는?",
            f"{case_title}에서 금융 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 금융 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 금융법 관련 판결은 무엇인가?",
        ],
        '해양': [
            f"{case_title}에서 해양법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 해양법 위반 사건의 판결 결과는?",
            f"{case_title}에서 해양 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 해양 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 해양법 관련 판결은 무엇인가?",
        ],
        '국제': [
            f"{case_title}에서 국제법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 국제법 위반 사건의 판결 결과는?",
            f"{case_title}에서 국제 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 국제 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 국제법 관련 판결은 무엇인가?",
        ],
        '보건': [
            f"{case_title}에서 보건법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 보건법 위반 사건의 판결 결과는?",
            f"{case_title}에서 보건 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 보건 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 보건법 관련 판결은 무엇인가?",
        ],
        '에너지': [
            f"{case_title}에서 에너지법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 에너지법 위반 사건의 판결 결과는?",
            f"{case_title}에서 에너지 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 에너지 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 에너지법 관련 판결은 무엇인가?",
        ],
        '군사': [
            f"{case_title}에서 군사법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 군사법 위반 사건의 판결 결과는?",
            f"{case_title}에서 군사 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 군사 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 군사법 관련 판결은 무엇인가?",
        ],
        '통신': [
            f"{case_title}에서 통신법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 통신법 위반 사건의 판결 결과는?",
            f"{case_title}에서 통신 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 통신 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 통신법 관련 판결은 무엇인가?",
        ],
        '교육': [
            f"{case_title}에서 교육법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 교육법 위반 사건의 판결 결과는?",
            f"{case_title}에서 교육 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 교육 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 교육법 관련 판결은 무엇인가?",
        ],
        '식품': [
            f"{case_title}에서 식품법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 식품법 위반 사건의 판결 결과는?",
            f"{case_title}에서 식품 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 식품 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 식품법 관련 판결은 무엇인가?",
        ],
        '항공': [
            f"{case_title}에서 항공법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 항공법 위반 사건의 판결 결과는?",
            f"{case_title}에서 항공 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 항공 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 항공법 관련 판결은 무엇인가?",
        ],
        '정보': [
            f"{case_title}에서 정보보호법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 정보보호법 위반 사건의 판결 결과는?",
            f"{case_title}에서 정보보호 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 정보보호 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 정보보호법 관련 판결은 무엇인가?",
        ],
        '미디어': [
            f"{case_title}에서 미디어법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 미디어법 위반 사건의 판결 결과는?",
            f"{case_title}에서 미디어 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 미디어 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 미디어법 관련 판결은 무엇인가?",
        ],
        '안전': [
            f"{case_title}에서 안전법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 안전법 위반 사건의 판결 결과는?",
            f"{case_title}에서 안전 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 안전 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 안전법 관련 판결은 무엇인가?",
        ],
        '공정거래': [
            f"{case_title}에서 공정거래법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 공정거래법 위반 사건의 판결 결과는?",
            f"{case_title}에서 공정거래 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 공정거래 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 공정거래법 관련 판결은 무엇인가?",
        ],
        '산업재해': [
            f"{case_title}에서 산업재해보상보험법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 산업재해보상보험법 위반 사건의 판결 결과는?",
            f"{case_title}에서 산업재해보상 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 산업재해보상 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 산업재해보상보험법 관련 판결은 무엇인가?",
        ],
        '보험': [
            f"{case_title}에서 보험법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 보험법 위반 사건의 판결 결과는?",
            f"{case_title}에서 보험 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 보험 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 보험법 관련 판결은 무엇인가?",
        ],
        '청소년': [
            f"{case_title}에서 청소년보호법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 청소년보호법 위반 사건의 판결 결과는?",
            f"{case_title}에서 청소년보호 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 청소년보호 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 청소년보호법 관련 판결은 무엇인가?",
        ],
        '공공기관': [
            f"{case_title}에서 공공기관운영법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 공공기관운영법 위반 사건의 판결 결과는?",
            f"{case_title}에서 공공기관운영 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 공공기관운영 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 공공기관운영법 관련 판결은 무엇인가?",
        ],
        '노인': [
            f"{case_title}에서 노인복지법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 노인복지법 위반 사건의 판결 결과는?",
            f"{case_title}에서 노인복지 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 노인복지 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 노인복지법 관련 판결은 무엇인가?",
        ],
        '아동': [
            f"{case_title}에서 아동복지법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 아동복지법 위반 사건의 판결 결과는?",
            f"{case_title}에서 아동복지 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 아동복지 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 아동복지법 관련 판결은 무엇인가?",
        ],
        '장애인': [
            f"{case_title}에서 장애인복지법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 장애인복지법 위반 사건의 판결 결과는?",
            f"{case_title}에서 장애인복지 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 장애인복지 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 장애인복지법 관련 판결은 무엇인가?",
        ],
        '동물': [
            f"{case_title}에서 동물보호법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 동물보호법 위반 사건의 판결 결과는?",
            f"{case_title}에서 동물보호 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 동물보호 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 동물보호법 관련 판결은 무엇인가?",
        ],
        '환경보호': [
            f"{case_title}에서 환경보호법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 환경보호법 위반 사건의 판결 결과는?",
            f"{case_title}에서 환경보호 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 환경보호 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 환경보호법 관련 판결은 무엇인가?",
        ],
        '금융': [
            f"{case_title}에서 금융법 위반에 대한 판결은 무엇인가?",
            f"{case_title}의 금융법 위반 사건의 판결 결과는?",
            f"{case_title}에서 금융 관련 법 위반에 대한 법원의 판단은?",
            f"{case_title}에서 금융 분쟁에 대한 판결은 무엇인가?",
            f"{case_title}의 금융법 관련 판결은 무엇인가?",
        ]
    }

    for keyword, pattern_list in patterns.items():
        if keyword in case_detail:
            return random.choice(pattern_list)
    
    return f"{case_title}에 대한 판결은 무엇인가?"

# 요약 함수
def summarize_text(text):
    inputs = summarizer_tokenizer([text], max_length=512, return_tensors="pt", truncation=True)
    summary_ids = summarizer_model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 각 행에 대해 요약, 질문 생성 및 답변 생성
results = []
for index, row in df.iterrows():
    context = row['court_detail']  # 적절한 컬럼 이름을 지정하세요
    summarized_context = summarize_text(context) if pd.notnull(context) else ""
    question = generate_question(row)

    # 토크나이저를 사용하여 입력 텍스트를 인코딩
    inputs = qa_tokenizer(question, summarized_context, return_tensors="pt")

    # 모델을 사용하여 답변 추론
    with torch.no_grad():
        outputs = qa_model(**inputs)

    # 답변의 시작과 끝 위치를 추론
    answer_start_index = torch.argmax(outputs.start_logits)
    answer_end_index = torch.argmax(outputs.end_logits) + 1

    # 인덱스를 사용하여 원본 컨텍스트에서 답변 추출
    answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start_index:answer_end_index]))

    results.append({
        "Question": question,
        "Context": summarized_context,
        "Answer": answer
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과를 엑셀 파일로 저장
output_path = 'data/qa_results.xlsx'
results_df.to_excel(output_path, index=False)


In [ ]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering
import torch

# 파일 경로
file_path = 'data/context.xlsx'

# 파일 로드
df = pd.read_excel(file_path)
df = df[['event_Title', 'court_detail']]

# 특수 문자 제거 및 공백 제거
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나로
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text.strip()

df['court_detail'] = df['court_detail'].apply(lambda x: clean_text(x) if pd.notnull(x) else '')

# KoBART 요약 모델과 토크나이저 불러오기 (예시, 실제 사용 시 KoBART 경로로 변경)
summarizer_tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("hyunwoongko/kobart")

# KoELECTRA QA 모델과 토크나이저 불러오기
qa_tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("monologg/koelectra-base-v3-finetuned-korquad")

# 각 판례의 내용을 기반으로 질문 생성
def generate_question(row):
    case_title = row.get('event_Title', '이 사건')
    case_detail = row.get('court_detail', '')

    keywords = ['저작권', '특허', '상표', '계약', '부동산', '노동', '형사', '민사', '세금', '회사', '환경', '교통', '가족', '상속', '지적재산권', '금융', '해양', '국제', '보건', '에너지', '군사', '통신', '교육', '식품', '항공', '정보', '미디어', '안전', '공정거래', '산업재해', '보험', '청소년', '공공기관', '노인', '아동', '장애인', '동물', '환경보호']

    for keyword in keywords:
        if keyword in case_detail:
            return f"{case_title}에서 {keyword}법 위반에 대한 판결은 무엇인가?"

    return f"{case_title}에 대한 판결은 무엇인가?"

# 요약 함수
def summarize_text(text):
    inputs = summarizer_tokenizer([text], max_length=512, return_tensors="pt", truncation=True)
    summary_ids = summarizer_model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 각 행에 대해 요약, 질문 생성 및 답변 생성
results = []
for index, row in df.iterrows():
    context = row['court_detail']
    summarized_context = summarize_text(context) if pd.notnull(context) else ""
    question = generate_question(row)
    
    print(f"Original Context: {context}")
    print(f"Summarized Context: {summarized_context}")
    print(f"Generated Question: {question}")

    if summarized_context:
        # 토크나이저를 사용하여 입력 텍스트를 인코딩
        inputs = qa_tokenizer(question, summarized_context, return_tensors="pt")

        # 모델을 사용하여 답변 추론
        with torch.no_grad():
            outputs = qa_model(**inputs)

        # 답변의 시작과 끝 위치를 추론
        answer_start_index = torch.argmax(outputs.start_logits)
        answer_end_index = torch.argmax(outputs.end_logits) + 1

        # 인덱스를 사용하여 원본 컨텍스트에서 답변 추출
        answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start_index:answer_end_index]))
    else:
        answer = ""

    print(f"Generated Answer: {answer}")
    results.append({
        "Question": question,
        "Context": summarized_context,
        "Answer": answer
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과를 엑셀 파일로 저장
output_path = 'data/qa_results.xlsx'
results_df.to_excel(output_path, index=False)

print(results_df)


In [ ]:
# 로컬 파일에서 데이터셋 불러오기
dataset = load_from_disk("data/dataset")

# RAG 모델과 토크나이저 불러오기
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained(
    "facebook/rag-token-nq",
    index_name="exact",
    use_dummy_dataset=False,
    passages_path=None,  # passages_path를 None으로 설정하고 아래에서 설정
    trust_remote_code=True
)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")

# 인덱싱할 데이터 준비
passages = dataset["text"]

# retriever에 passages 추가
retriever.index.index_data(passages)

In [ ]:
# 문서 임베딩을 저장한 파일 로드
embeddings_df = pd.read_excel('data/embeddings_data.xlsx')
df = pd.read_excel('data/law_recomended.xlsx')

In [ ]:
retriever.index.add(embeddings_df['court_detail'].tolist())

In [ ]:
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re


my_id = 'realstone.in'
org = '400201' # 대법원
JO = ['민사', '형사'] #%EB%AF%BC%EB%B2%95
base_url = f'https://www.law.go.kr/DRF/lawSearch.do?OC={my_id}&target=prec&type=XML&display=100'
event_base_url = 'http://www.law.go.kr'

'''
https://www.law.go.kr/DRF/lawSearch.do?OC=realstone.in&target=prec&type=XML&display=100&page=1&search=3&org=400201&JO=%EB%AF%BC%EB%B2%95
'''

# 사건명 사건번호 사건종류명 판례일련번호 판례상세링크 법원명


page=1
url = f'{base_url}&page={page}'#&org={org}''&JO={JO}
event_Title = []
event_num = []
event_type = []
jPrecedent_num = []
jPrecedent_link_get = []
jPrecedent_link_repl = []
court_name = []
court_num = []
court_detail = []
court_reference = []
court_Decision = []

response = requests.get(url)
title_XML = response.text
title_soup = BeautifulSoup(title_XML, 'xml') 
total_page = int(title_soup.select('totalCnt')[0].string)//100+1

for page in tqdm(range(5000)) : #range(1, total_page+1)) :
    url = f'{base_url}&page={page}'
    response = requests.get(url)
    title_XML = response.text
    title_soup = BeautifulSoup(title_XML, 'xml') 

    for i in range(len(title_soup.select('사건명'))) :
        event_Title.append(title_soup.select('사건명')[i].string)
        event_num.append(title_soup.select('사건번호')[i].string)
        event_type.append(title_soup.select('사건종류명')[i].string)
        jPrecedent_num.append(title_soup.select('판례일련번호')[i].string)
        jPrecedent_link_get.append(title_soup.select('판례상세링크')[i].string)
        court_name.append(title_soup.select('법원명')[i].string)


for i in tqdm(range(len(jPrecedent_link_get))) :
    jPrecedent_link_repl.append(jPrecedent_link_get[i][:-10].replace('HTML', 'XML'))
    event_url = f'{event_base_url}{jPrecedent_link_repl[i]}'
    event_response = requests.get(event_url)
    main_XMLevent = event_response.text
    main_soup = BeautifulSoup(main_XMLevent, 'xml')
    #print(event_url)
    #court_num.append(main_soup.select('사건번호')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_detail.append(main_soup.select('판례내용')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_reference.append(main_soup.select('참조조문')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_Decision.append(main_soup.select('판시사항')[0].string.replace('</br>', '').replace('<br/>', ''))


df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_excel('data/court_all.xlsx')

In [ ]:
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

df = pd.read_csv('court_title.csv', encoding='cp949')
event_Title = list(df['event_Title'][:4999])
event_num = list(df['event_num'][:4999])
event_type = list(df['event_type'][:4999])
jPrecedent_num = list(df['jPrecedent_num'][:4999])
jPrecedent_link_get = list(df['jPrecedent_link_get'][:4999])
court_name = list(df['court_name'][:4999])

df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, \
                  'court_num':court_num, 'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})
#df1 = pd.DataFrame()

df.to_excel('court_title.xlsx')
#df1.to_csv('court_detail.csv', encoding='cp949')

In [ ]:
df = pd.read_csv('court_title.csv', encoding='cp949')
event_Title = list(df['event_Title'][:5000])
event_num = list(df['event_num'][:5000])
event_type = list(df['event_type'][:5000])
jPrecedent_num = list(df['jPrecedent_num'][:5000])
jPrecedent_link_get = list(df['jPrecedent_link_get'][:5000])
court_name = list(df['court_name'][:5000])
df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_excel('data/court_all.xlsx')

In [ ]:
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

df = pd.read_csv('court_title.csv', encoding='cp949')
event_Title = list(df['event_Title'][:5000])
event_num = list(df['event_num'][:5000])
event_type = list(df['event_type'][:5000])
jPrecedent_num = list(df['jPrecedent_num'][:5000])
jPrecedent_link_get = list(df['jPrecedent_link_get'][:5000])
court_name = list(df['court_name'][:5000])

jPrecedent_link_repl = []

court_detail = []
court_reference = []
court_Decision = []

for i in tqdm(range(5000)) :
    jPrecedent_link_repl.append(jPrecedent_link_get[i][:-10].replace('HTML', 'XML'))
    event_url = f'{event_base_url}{jPrecedent_link_repl[i]}'
    event_response = requests.get(event_url)
    main_XMLevent = event_response.text
    main_soup = BeautifulSoup(main_XMLevent, 'xml')
    #print(event_url)
    #court_num.append(main_soup.select('사건번호')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_detail.append(main_soup.select('판례내용')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_reference.append(main_soup.select('참조조문')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_Decision.append(main_soup.select('판시사항')[0].string.replace('</br>', '').replace('<br/>', ''))
    
df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_excel('data/court_all.xlsx')

In [ ]:
df = pd.DataFrame({'event_Title':event_Title[:5000], 'event_num':event_num[:5000], 'event_type':event_type[:5000], 'jPrecedent_num':jPrecedent_num[:5000], \
                  'jPrecedent_link_get':jPrecedent_link_get[:5000], 'court_name':court_name, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_excel('data/court_all.xlsx')

In [ ]:
#!pip install transformers
#!pip install torch

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tqdm.notebook import tqdm

df = pd.read_csv('court_test.csv', encoding='CP949')
text = df['court_detail'].replace('\n', '').tolist()

# KC-BERT 모델과 토크나이저 로드
model_name = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 문장 임베딩 함수 정의
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] 토큰의 출력 사용
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.detach().numpy()

embedding = [get_sentence_embedding(i) for i in tqdm(text)]

    

In [ ]:
#!pip install elasticsearch
#!pip install datasets
#!pip install faiss-cpu

In [ ]:
from transformers import RagRetriever, RagTokenizer, RagSequenceForGeneration, DPRContextEncoder, DPRContextEncoderTokenizer
import pandas as pd
from datasets import Dataset, load_from_disk
import torch

# 판례 데이터 로드
df = pd.read_csv('court_test.csv', encoding='CP949')
df['title'] = df['event_Title']  # 'court_title' 컬럼을 'title'로 사용
df['text'] = df['court_detail']
documents = df[['title', 'text']].to_dict(orient='records')  # 'title'과 'text' 컬럼을 포함하는 리스트로 변환

# 데이터셋으로 변환
dataset = Dataset.from_dict({
    "title": [doc['title'] for doc in documents],
    "text": [doc['text'] for doc in documents]
})

# DPRContextEncoder 모델과 토크나이저 로드
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

# 최대 길이 설정 (512 토큰)
max_length = context_tokenizer.model_max_length

# 텍스트 임베딩 생성 함수
def embed_texts(batch):
    inputs = context_tokenizer(batch['text'], truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    with torch.no_grad():
        embeddings = context_encoder(**inputs).pooler_output
    batch['embeddings'] = embeddings.numpy().tolist()  # NumPy 배열을 리스트로 변환
    return batch

# 텍스트 임베딩 생성 및 데이터셋에 추가
dataset = dataset.map(embed_texts, batched=True, batch_size=8)


In [ ]:
# 데이터셋 저장
dataset_path = 'dataset'
dataset.save_to_disk(dataset_path)

# FAISS 인덱스 생성 및 저장
dataset.add_faiss_index(column='embeddings')
index_path = 'index'
dataset.get_index('embeddings').save(index_path)


In [ ]:
# RAG 모델과 토크나이저 로드
tokenizer = RagTokenizer.from_pretrained('facebook/rag-sequence-nq')
model = RagSequenceForGeneration.from_pretrained('facebook/rag-sequence-nq')

# RAG Retriever 설정
retriever = RagRetriever.from_pretrained(
    'facebook/rag-sequence-nq',
    index_name="custom",
    passages_path=dataset_path,  # passages_path를 dataset_path로 설정
    index_path=index_path  # index_path 설정
)

# 사용자 질문에 대한 답변 생성
query = input("질문을 입력하세요: ")
input_ids = tokenizer(query, return_tensors='pt')['input_ids']

try:
    generated = model.generate(input_ids, retriever=retriever)
    print(tokenizer.decode(generated[0], skip_special_tokens=True))
except Exception as e:
    print(f"오류 발생: {e}")


In [ ]:
# 사용자 질문에 대한 답변 생성
query = input("질문을 입력하세요: ")
input_ids = tokenizer(query, return_tensors='pt')['input_ids']

try:
    generated = model.generate(input_ids, retriever=retriever)
    print(tokenizer.decode(generated[0], skip_special_tokens=True))
except Exception as e:
    print(f"오류 발생: {e}")

In [ ]:
# 데이터셋 로드 확인
loaded_dataset = load_from_disk(dataset_path)
print(loaded_dataset)

# 인덱스 로드 확인
loaded_index = loaded_dataset.get_index('embeddings')
print(loaded_index)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import pandas as pd
import numpy as np
import pickle
import csv
from tqdm import tqdm

# 데이터 로드 및 전처리
df = pd.read_csv('/content/court.csv', encoding='CP949')
komoran = Komoran()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = [komoran.morphs(sentence) for sentence in tqdm(text, desc="텍스트 형태소 단위로 나누기")]

# 전처리된 텍스트 저장
with open('/content/court_processed_text.pkl', 'wb') as f:
    pickle.dump(processed_text, f)

# Word2Vec 모델 학습
model = Word2Vec(processed_text, vector_size=100, window=5, min_count=1, workers=4)
model.save('/content/civil_word2vec_model.bin')

# TF-IDF 벡터화
tfidf = TfidfVectorizer(tokenizer=komoran.morphs, max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['court_detail'])

# 문장 벡터 생성 함수
def get_sentence_vector(sentence, model, tfidf):
    words = komoran.morphs(sentence)
    word_vectors = []
    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word] * tfidf.idf_[tfidf.vocabulary_.get(word, 0)])
    
    if not word_vectors:
        return np.zeros(model.vector_size)
    
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector

# 전체 문서 벡터 생성 및 저장
sentence_vectors = np.array([get_sentence_vector(sentence, model, tfidf) for sentence in tqdm(df['court_detail'], desc="문서 벡터 생성")])
with open('/content/sentence_vectors.pkl', 'wb') as f:
    pickle.dump(sentence_vectors, f)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import pandas as pd
import numpy as np
import pickle
import csv
from tqdm import tqdm


# 저장된 모델과 매트릭스 불러오기
with open('civil_law/civil_word2vec_model.bin', 'rb') as f:
    word2vec_model = pickle.load(f)

with open('civil_law/court_processed_text.pkl', 'rb') as f:
    court_processed_text = pickle.load(f)


    
# 데이터 로드 및 전처리
df = pd.read_csv('civil_law/court.csv', encoding='CP949')
komoran = Komoran()
text = df['court_detail'].str.replace("\n", '')


# TF-IDF 벡터화
tfidf = TfidfVectorizer(tokenizer=komoran.morphs, max_features=5000)
tfidf_matrix = tfidf.fit_transform(text)

# 문장 벡터 생성 함수
def get_sentence_vector(sentence, model, tfidf):
    words = komoran.morphs(sentence)
    word_vectors = []
    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word] * tfidf.idf_[tfidf.vocabulary_.get(word, 0)])
    
    if not word_vectors:
        return np.zeros(model.vector_size)
    
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector

# 전체 문서 벡터 생성 및 저장
sentence_vectors = np.array([get_sentence_vector(sentence, model, tfidf) for sentence in tqdm(df['court_detail'], desc="문서 벡터 생성")])
with open('/content/sentence_vectors.pkl', 'wb') as f:
    pickle.dump(sentence_vectors, f)

In [ ]:
# 문장 벡터 생성 함수
def get_sentence_vector(sentence, model, tfidf):
    words = komoran.morphs(sentence)
    word_vectors = []
    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word] * tfidf.idf_[tfidf.vocabulary_.get(word, 0)])
    
    if not word_vectors:
        return np.zeros(model.vector_size)
    
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector


# 데이터 로드 및 전처리
df = pd.read_csv('civil_law/court.csv', encoding='CP949')
komoran = Komoran()
text = df['court_detail'].str.replace("\n", '')

#tfidf_vocab = tfidf_vectorizer.vocabulary_
#tfidf_idf = tfidf_vectorizer.idf_


# 전체 문서 벡터 생성 및 저장
sentence_vectors = np.array([get_sentence_vector(sentence, word2vec_model, tfidf) for sentence in tqdm(df['court_detail'], desc="문서 벡터 생성")])
with open('path/to/sentence_vectors.pkl', 'wb') as f:
    pickle.dump(sentence_vectors, f)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.spatial.distance import jaccard
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import pandas as pd
import numpy as np
import pickle

# 데이터 로드 및 전처리
df = pd.read_csv('./court.csv', encoding='cp949')
komoran = Komoran()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = [komoran.morphs(sentence) for sentence in text]

# TF-IDF 벡터라이저 및 매트릭스 생성
tfidf_vectorizer_unigram = TfidfVectorizer(ngram_range=(1, 1))
tfidf_matrix_unigram = tfidf_vectorizer_unigram.fit_transform([' '.join(sentence) for sentence in processed_text])

tfidf_vectorizer_bigram = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix_bigram = tfidf_vectorizer_bigram.fit_transform([' '.join(sentence) for sentence in processed_text])

# Word2Vec 모델 학습
word2vec_model = Word2Vec(sentences=processed_text, vector_size=100, window=5, min_count=2, workers=4)

# 문장 임베딩 생성
def get_sentence_embedding(model, sentence):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

sentence_embeddings = np.array([get_sentence_embedding(word2vec_model, sentence) for sentence in processed_text])

# 유사도 계산 함수
def calculate_similarity(matrix, method='cosine'):
    if method == 'cosine':
        return cosine_similarity(matrix)
    elif method == 'euclidean':
        return -euclidean_distances(matrix)  # 유사도는 거리가 작을수록 크기 때문에 음수로 변환
    elif method == 'jaccard':
        similarities = np.zeros((matrix.shape[0], matrix.shape[0]))
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[0]):
                if i != j:
                    similarities[i, j] = 1 - jaccard(matrix[i].toarray()[0], matrix[j].toarray()[0])
        return similarities
    else:
        raise ValueError(f"Unknown method: {method}")

# 유사도 계산
tfidf_unigram_similarity_cosine = calculate_similarity(tfidf_matrix_unigram, 'cosine')
tfidf_bigram_similarity_cosine = calculate_similarity(tfidf_matrix_bigram, 'cosine')
word2vec_similarity_cosine = calculate_similarity(sentence_embeddings, 'cosine')

tfidf_unigram_similarity_euclidean = calculate_similarity(tfidf_matrix_unigram, 'euclidean')
tfidf_bigram_similarity_euclidean = calculate_similarity(tfidf_matrix_bigram, 'euclidean')
word2vec_similarity_euclidean = calculate_similarity(sentence_embeddings, 'euclidean')

tfidf_unigram_similarity_jaccard = calculate_similarity(tfidf_matrix_unigram, 'jaccard')
tfidf_bigram_similarity_jaccard = calculate_similarity(tfidf_matrix_bigram, 'jaccard')
# Jaccard 유사도는 Word2Vec 임베딩에는 사용하지 않음 (이진 데이터에 적합)

# 가장 높은 평균 유사도를 가지는 방법 선택
similarities = {
    'TF-IDF Unigram Cosine': np.mean(tfidf_unigram_similarity_cosine),
    'TF-IDF Bigram Cosine': np.mean(tfidf_bigram_similarity_cosine),
    'Word2Vec Cosine': np.mean(word2vec_similarity_cosine),
    'TF-IDF Unigram Euclidean': np.mean(tfidf_unigram_similarity_euclidean),
    'TF-IDF Bigram Euclidean': np.mean(tfidf_bigram_similarity_euclidean),
    'Word2Vec Euclidean': np.mean(word2vec_similarity_euclidean),
    'TF-IDF Unigram Jaccard': np.mean(tfidf_unigram_similarity_jaccard),
    'TF-IDF Bigram Jaccard': np.mean(tfidf_bigram_similarity_jaccard)
}

best_method = max(similarities, key=similarities.get)
best_similarity_matrix = None

if 'Unigram' in best_method:
    best_model = tfidf_vectorizer_unigram
    if 'Cosine' in best_method:
        best_similarity_matrix = tfidf_unigram_similarity_cosine
    elif 'Euclidean' in best_method:
        best_similarity_matrix = tfidf_unigram_similarity_euclidean
    elif 'Jaccard' in best_method:
        best_similarity_matrix = tfidf_unigram_similarity_jaccard
elif 'Bigram' in best_method:
    best_model = tfidf_vectorizer_bigram
    if 'Cosine' in best_method:
        best_similarity_matrix = tfidf_bigram_similarity_cosine
    elif 'Euclidean' in best_method:
        best_similarity_matrix = tfidf_bigram_similarity_euclidean
    elif 'Jaccard' in best_method:
        best_similarity_matrix = tfidf_bigram_similarity_jaccard
elif 'Word2Vec' in best_method:
    best_model = word2vec_model
    if 'Cosine' in best_method:
        best_similarity_matrix = word2vec_similarity_cosine
    elif 'Euclidean' in best_method:
        best_similarity_matrix = word2vec_similarity_euclidean

print(f"Best method: {best_method}")

# 모델과 유사도 매트릭스 저장
with open('best_model_civil_with_komoran.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('best_similarity_matrix_civil_with_komoran.pkl', 'wb') as f:
    pickle.dump(best_similarity_matrix, f)

# 전처리된 텍스트 저장
with open('court_processed_text_civil_with_komoran.pkl', 'wb') as f:
    pickle.dump(processed_text, f)


In [ ]:
!pip install mecab-python3

코드 설명

데이터 로드 및 전처리:

데이터를 로드하고 줄바꿈 문자를 제거합니다.

텍스트를 형태소 단위로 나눕니다.

TF-IDF 벡터라이저 및 매트릭스 생성:

1-그램 및 2-그램을 사용하여 TF-IDF 매트릭스를 생성합니다.

Word2Vec 모델 학습 및 문장 임베딩 생성:

Word2Vec 모델을 학습하고, 각 문장을 임베딩 벡터로 변환합니다.

유사도 계산 함수:

주어진 매트릭스에 대해 코사인 유사도를 계산합니다.

유사도 계산:

각각의 방법에 대해 유사도를 계산합니다.

가장 높은 평균 유사도를 가지는 방법 선택:

각 방법의 평균 유사도를 계산하고, 가장 높은 유사도를 가지는 방법을 선택합니다.

선택된 모델과 유사도 매트릭스 저장:

선택된 모델과 유사도 매트릭스를 저장합니다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from konlpy.tag import Komoran
import pandas as pd
import numpy as np
import pickle

#mecab = Mecab('C:/mecab/mecab-ko-dic')

# 데이터 로드 및 전처리
df = pd.read_csv('civil law/court.csv', encoding='cp949')
komoran = Komoran()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = []
for i in text :
    processed_text.append(komoran.morphs(i))
#= [komoran.morphs(sentence) for sentence in text]

# 전처리된 텍스트 저장
with open('court_processed_text.pkl', 'wb') as f:
    pickle.dump(processed_text, f)

model = Word2Vec(processed_text, vector_size=100, window=5, min_count=1, workers=4)

model.save('civil_word2vec_model.bin')





In [ ]:
import pickle
import numpy as np
from konlpy.tag import Komoran
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

def get_sentence_vector(sentence, model):
    # 문장을 토큰화하여 단어 리스트를 얻습니다.
    komoran = Komoran()
    words = komoran.morphs(sentence)
    # 단어 리스트에서 각 단어의 벡터를 가져옵니다.
    word_vectors = []
    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    
    # 단어 벡터가 없다면, 제로 벡터를 반환합니다.
    if not word_vectors:
        return np.zeros(model.vector_size)

    # 단어 벡터들의 평균을 계산하여 문장 벡터를 얻습니다.
    sentence_vector = np.mean(word_vectors, axis=0)
    
    return sentence_vector


# 저장된 모델과 매트릭스 불러오기
with open('civil_word2vec_model.bin', 'rb') as f:
    word2vec_model = pickle.load(f)

with open('court_processed_text.pkl', 'rb') as f:
    court_processed_text = pickle.load(f)

# 처리된 텍스트가 리스트 형태라면, 이를 문자열로 변환
processed_sentences = [' '.join(sentence) if isinstance(sentence, list) else sentence for sentence in tqdm(court_processed_text)]

# 처리된 텍스트를 문장 벡터로 변환
sentence_vectors = np.array([get_sentence_vector(sentence, word2vec_model) for sentence in tqdm(processed_sentences)])



In [ ]:
import pickle
with open('sentence_vectors.pkl', 'wb') as f:
    pickle.dump(sentence_vectors, f)

In [ ]:
import numpy as np
from konlpy.tag import Komoran
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle



#===========================================함수========================================

def get_sentence_vector(sentence, model):
    # 문장을 토큰화하여 단어 리스트를 얻습니다.
    komoran = Komoran()
    words = komoran.morphs(sentence)
    # 단어 리스트에서 각 단어의 벡터를 가져옵니다.
    word_vectors = []
    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    
    # 단어 벡터가 없다면, 제로 벡터를 반환합니다.
    if not word_vectors:
        return np.zeros(model.vector_size)

    # 단어 벡터들의 평균을 계산하여 문장 벡터를 얻습니다.
    sentence_vector = np.mean(word_vectors, axis=0)
    
    return sentence_vector

def recommend_court(input_sentence, top_n=5):    
    # 입력 문장을 TF-IDF 변환
    input_vector = get_sentence_vector(input_sentence, word2vec_model)
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([input_vector], sentence_vectors)[0]
    
    # 유사도 순으로 정렬된 상위 n개 인덱스 추출
    most_similar_indices = np.argsort(similarities)[-top_n:][::-1]
    
    # 유사도 0인 경우 처리
    recommendations = []
    for idx in most_similar_indices:
        if similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], \
                                df.iloc[idx]['event_num'], \
                                df.iloc[idx]['event_type'], \
                                df.iloc[idx]['jPrecedent_num'], \
                                df.iloc[idx]['court_name'], \
                                df.iloc[idx]['court_num'], \
                                df.iloc[idx]['court_detail'], \
                                df.iloc[idx]['court_reference'], \
                                df.iloc[idx]['court_Decision'], \
                                similarities[idx]*100))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations
    
    
    
#===========================================파일========================================
    
with open('civil_word2vec_model.bin', 'rb') as f:
    word2vec_model = pickle.load(f)
    
with open('sentence_vectors.pkl', 'rb') as f:
    sentence_vectors = pickle.load(f)

df = pd.read_csv('civil_law\court.csv', encoding='cp949')
 
    
    
#===========================================실행========================================

court_input = str(input('상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :'))
court_detail_OX = str(input('판례 및 판시 사항을 보시겠습니까?(Y/N) : '))
input_sentence = court_input

# 판례
recommended_court = recommend_court(input_sentence)
if isinstance(recommended_court, str):
    print(recommended_court)
else:
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_num, court_detail, court_reference, court_Decision, similarity  \
    in recommended_court:
        print('==================================================================================================================')
        if (court_detail_OX == 'N') or (court_detail_OX == 'n') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n사건 번호 : {court_num} \n\n참조조문 \n{court_reference}\n\n\n")
        else :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n판례 내용 :\n{court_detail} \n\n참조조문 \n{court_reference} \
                    \n\n판시사항 :\n{court_Decision}\n\n\n")
        print('==================================================================================================================')

In [3]:
import pandas as pd
print(pd.__version__)

AttributeError: module 'pandas' has no attribute '__version__'

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 로드
data = pd.read_excel('./data/law_recomended.xlsx') # 추출한 텍스트 데이터 파일

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['court_detail'])

# K-Means 군집화
kmeans = KMeans(n_clusters=7, random_state=42) # 원하는 군집 수 설정
data['cluster'] = kmeans.fit_predict(X)



AttributeError: module 'pandas' has no attribute 'read_excel'

In [ ]:
def recommend_court(input_sentence, top_n=5):
    # 입력 문장을 Okt를 사용하여 전처리
    okt = Okt()
    processed_input = ' '.join(okt.morphs(input_sentence))
    
    # 입력 문장을 TF-IDF 변환
    input_tfidf = tfidf_vectorizer.transform([processed_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    
    # 유사도 순으로 정렬된 상위 n개 인덱스 추출
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 유사도 0인 경우 처리
    recommendations = []
    for idx in top_indices:
        if cosine_similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], \
                                df.iloc[idx]['event_num'], \
                                df.iloc[idx]['event_type'], \
                                df.iloc[idx]['jPrecedent_num'], \
                                df.iloc[idx]['court_name'], \
                                df.iloc[idx]['court_num'], \
                                df.iloc[idx]['court_detail'], \
                                df.iloc[idx]['court_reference'], \
                                df.iloc[idx]['court_Decision'], \
                                cosine_similarities[idx]*100))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations

court_input = str(input('상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :'))
court_detail_OX = str(input('판례 및 판시 사항을 보시겠습니까?(Y/N) : '))
input_sentence = court_input

# 판례
recommended_court = recommend_court(input_sentence)
if isinstance(recommended_court, str):
    print(recommended_court)
else:
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_num, court_detail, court_reference, court_Decision, similarity  \
    in recommended_court:
        
        if (court_detail_OX == 'N') or (court_detail_OX == 'n') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n사건 번호 : {court_num} \n\n참조조문 \n{court_reference}\n\n\n")
        else :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n판례 내용 :\n{court_detail} \n\n참조조문 \n{court_reference} \
                    \n\n판시사항 :\n{court_Decision}\n\n\n")

# API 제공 판례 크롤링 (전체 )

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import requests
import re

my_id = 'realstone.in'
#org = '400201' # 대법원
#JO = '민법' #%EB%AF%BC%EB%B2%95
base_url = f'https://www.law.go.kr/DRF/lawSearch.do?OC={my_id}&target=prec&type=XML&display=100'
event_base_url = 'http://www.law.go.kr'

'''
https://www.law.go.kr/DRF/lawSearch.do?OC=realstone.in&target=prec&type=XML&display=100&page=1&search=3&org=400201&JO=%EB%AF%BC%EB%B2%95
'''

# 사건명 사건번호 사건종류명 판례일련번호 판례상세링크 법원명
event_Title = []
event_num = []
event_type = []
jPrecedent_num = []
jPrecedent_link_get = []
jPrecedent_link_repl = []
court_name = []
court_num = []
court_detail = []
court_reference = []
court_Decision = []
page=1
url = f'{base_url}&page={page}'
#&org={org}
response = requests.get(url)
title_XML = response.text
title_soup = BeautifulSoup(title_XML, 'xml') 

total_page = int(title_soup.select('totalCnt')[0].string)//100+1

for page in tqdm(range(1, total_page+1)) :
    url = f'{base_url}&page={page}'
    response = requests.get(url)
    title_XML = response.text
    title_soup = BeautifulSoup(title_XML, 'xml') 

    for i in range(len(title_soup.select('사건명'))) :
        event_Title.append(title_soup.select('사건명')[i].string)
        event_num.append(title_soup.select('사건번호')[i].string)
        event_type.append(title_soup.select('사건종류명')[i].string)
        jPrecedent_num.append(title_soup.select('판례일련번호')[i].string)
        jPrecedent_link_get.append(title_soup.select('판례상세링크')[i].string)
        court_name.append(title_soup.select('법원명')[i].string)
        

for i in tqdm(range(len(jPrecedent_link_get))) :
    jPrecedent_link_repl.append(jPrecedent_link_get[i][:-10].replace('HTML', 'XML'))
    event_url = f'{event_base_url}{jPrecedent_link_repl[i]}'
    event_response = requests.get(event_url)
    main_XMLevent = event_response.text
    main_soup = BeautifulSoup(main_XMLevent, 'xml')
    #print(event_url)
    court_num.append(main_soup.select('사건번호')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_detail.append(main_soup.select('판례내용')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_reference.append(main_soup.select('참조조문')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_Decision.append(main_soup.select('판시사항')[0].string.replace('</br>', '').replace('<br/>', ''))

    
df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, 'court_num':court_num, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_excel('data/court_all_data_0712.xlsx')

  0%|          | 0/875 [00:00<?, ?it/s]

  0%|          | 0/87446 [00:00<?, ?it/s]

AttributeError: module 'pandas' has no attribute 'DataFrame'

In [8]:
df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, 'court_num':court_num, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_excel('data/court_all_data_0712.xlsx')

C:\Users\reals\AppData\Local\Temp\ipykernel_14192\4205941862.py:5: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel('data/court_all_data_0712.xlsx')


In [6]:
!pip install --upgrade --force-reinstall pandas

  Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl.metadata (19 kB)
     ---------------------------------------- 60.9/60.9 kB 3.2 MB/s eta 0:00:00
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl (11.6 MB)
   ---------------------------------------- 16.5/16.5 MB 38.6 MB/s eta 0:00:00
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
   --------------------------------------- 505.5/505.5 kB 16.0 MB/s eta 0:00:00
   --------------------------------------- 345.4/345.4 kB 20.9 MB/s eta 0:00:00
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.3
    Uninstalling pytz-2021.3:
      Successfully uninstalled pytz-2021.3
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2024.1
    Uninstalling tzdata-2024.1:
      Successfully unins

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
faiss-cpu 1.8.0.post1 requires numpy<2.0,>=1.0, but you have numpy 2.0.0 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.0 which is incompatible.
scipy 1.8.0 requires numpy<1.25.0,>=1.17.3, but you have numpy 2.0.0 which is incompatible.
tensorflow-intel 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.1

In [7]:
import sys
import pandas as pd
sys.modules.pop('pandas', None)
import pandas as pd

import pandas as pd

# 데이터프레임 생성
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

print(df)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\reals\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\reals\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\reals\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\reals\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\reals\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\reals\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\reals\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\reals\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\reals\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\reals\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

   A  B
0  1  4
1  2  5
2  3  6


In [ ]:
import pandas as pd
'''
event_Title 
event_num 
event_type 
jPrecedent_num 
jPrecedent_link_get 
court_name 
court_num 
court_detail 
court_reference 
court_Decision
'''

df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, 'court_num':court_num, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_csv('court.csv',encoding='cp949')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import pandas as pd
import pickle

df = pd.read_csv('court.csv', encoding='cp949')
# 줄거리 텍스트 데이터 전처리
okt = Okt()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = [' '.join(okt.morphs(sentence)) for sentence in text]

# TF-IDF 벡터라이저 초기화 및 학습
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_text)

# 모델과 TF-IDF 매트릭스 저장
with open('court_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('court_tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# 전처리된 텍스트 저장
with open('court_processed_text.pkl', 'wb') as f:
    pickle.dump(processed_text, f)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import pandas as pd
import pickle


df = pd.read_csv('court.csv', encoding='cp949')

# 저장된 모델과 매트릭스 불러오기
with open('civil law/court_tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('civil law/court_tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

with open('civil law/court_processed_text.pkl', 'rb') as f:
    processed_text = pickle.load(f)

def recommend_court(input_sentence, top_n=5):
    # 입력 문장을 Okt를 사용하여 전처리
    okt = Okt()
    processed_input = ' '.join(okt.morphs(input_sentence))
    
    # 입력 문장을 TF-IDF 변환
    input_tfidf = tfidf_vectorizer.transform([processed_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    
    # 유사도 순으로 정렬된 상위 n개 인덱스 추출
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 유사도 0인 경우 처리
    recommendations = []
    for idx in top_indices:
        if cosine_similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], \
                                df.iloc[idx]['event_num'], \
                                df.iloc[idx]['event_type'], \
                                df.iloc[idx]['jPrecedent_num'], \
                                df.iloc[idx]['court_name'], \
                                df.iloc[idx]['court_num'], \
                                df.iloc[idx]['court_detail'], \
                                df.iloc[idx]['court_reference'], \
                                df.iloc[idx]['court_Decision'], \
                                cosine_similarities[idx]*100))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations

court_input = str(input('상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :'))
court_detail_OX = str(input('판례 및 판시 사항을 보시겠습니까?(Y/N) : '))
input_sentence = court_input

# 판례
recommended_court = recommend_court(input_sentence)
if isinstance(recommended_court, str):
    print(recommended_court)
else:
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_num, court_detail, court_reference, court_Decision, similarity  \
    in recommended_court:
        
        if (court_detail_OX == 'N') or (court_detail_OX == 'n') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n사건 번호 : {court_num} \n\n참조조문 \n{court_reference}\n\n\n")
        else :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f}%)\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n판례 내용 :\n{court_detail} \n\n참조조문 \n{court_reference} \
                    \n\n판시사항 :\n{court_Decision}\n\n\n")


soup

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from konlpy.tag import Okt
import pandas as pd
import pickle  # 모델 저장 및 로딩을 위해
import numpy as np
import nltk

def model(filepath, court_detail):
    tfidf = TfidfVectorizer(tokenizer=Okt().morphs, max_features=500)
    tfidf.fit(court_detail)
    # 모델 저장
    with open(filepath, 'wb') as f:
        pickle.dump(tfidf, f)
'''
TfidfVectorizer() 하이퍼 파라미터
    min_df : 설정한 값보다 특정 Token의 df 값이 더 적게 나오면 벡터화 과정에서 제거
    anlayzer : 분석 단위를 의미, ‘word’의 경우 간어 하나를 단위로, ‘char’는 문자 하나를 단위로
    sublinear_tf : 문서의 단어 빈도수(tf:term frequency)에 대한 smoothing 여부를 설정
    ngram_range : 빈도의 기본 단위를 어떤 범위의 n-gram으로 설정할 것인지를 보는 인자
    max_features : 각 벡터의 최대 길이(특징의 길이)를 설정
'''
        
        
# 저장된 TF-IDF 모델을 불러와 사용하는 함수
def load_tfidf_model(filepath='tfidf_model.pkl'):
    with open(filepath, 'rb') as f:
        return pickle.load(f)
    
def model_with_pretrained_tfidf(detail, num, court_detail):
    tfidf = load_tfidf_model()  # 저장된 모델 불러오기
    all_detail = court_detail + [detail]
    tfidf_matrix = tfidf.transform(all_detail)  # fit_transform 대신 transform 사용
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    cosine_similar_indices = np.argsort(cosine_sim)[-num:][::-1]
    return cosine_similar_indices

court_detail = pd.read_csv('court.csv', encoding='cp949')['court_detail']
path = 'tfidf_model.pkl'
model(path, court_detail)

'''
detail = str(input('내용을 입력해 주세요 : '))#'새로 선임된 청산인을 기존 이사회가 해임시킬 수 있어?'
num = int(input('몇 개의 판례를 보시겠습니까? (최대 5개) : '))
if num == 1:
    OX = str(input('판례 내용을 보시겠습니까?(O, X) : '))
    print(f'질문 : "{detail}"')
    index = model_with_pretrained_tfidf(detail, num, court_detail)
    print(f'사건 번호 : {court_num[index[0][0]]}')
    print(f'참조 조문 : {court_reference[index[0][0]]}')
    print(f'판시 사항 : {court_Decision[index[0][0]]}')
    if OX == 'O' :
        print(f'유사한 판례 : {court_detail[index[0][0]]}')
elif num <= 5:
    OX = str(input('판례 내용을 보시겠습니까?(O, X) : '))
    print(f'질문 : "{detail}"')
    index = model_with_pretrained_tfidf(detail, num, court_detail)
    for i in index[0][:num]:
        print(f'사건 번호 : {court_num[index[0][i]]}')
        print(f'참조 조문 : {court_reference[index[0][i]]}')
        print(f'판시 사항 : {court_Decision[index[0][i]]}')
        if OX == 'O' :
            print(f'유사한 판례 : {court_detail[index[0][i]]}')
elif num > 5:
    OX = str(input('판례 내용을 보시겠습니까?(O, X) : '))
    print(f'질문 : "{detail}"')
    index = model_with_pretrained_tfidf(detail, 5, court_detail)
    for i in index[0][:5] :
        print(f'사건 번호 : {court_num[index[0][i]]}')
        print(f'참조 조문 : {court_reference[index[0][i]]}')
        print(f'판시 사항 : {court_Decision[index[0][i]]}')
        if OX == 'O' :
            print(f'유사한 판례 : {court_detail[index[0][i]]}')
'''

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import requests
import re

my_id = 'realstone.in'
org = '400201' # 대법원
JO = '근로' #%EB%AF%BC%EB%B2%95
base_url = f'https://www.law.go.kr/DRF/lawSearch.do?OC={my_id}&target=prec&type=XML&display=100'
event_base_url = 'http://www.law.go.kr'

'''
https://www.law.go.kr/DRF/lawSearch.do?OC=realstone.in&target=prec&type=XML&display=100&page=1&search=3&org=400201&JO=%EB%AF%BC%EB%B2%95
'''

# 사건명 사건번호 사건종류명 판례일련번호 판례상세링크 법원명
event_Title = []
event_num = []
event_type = []
jPrecedent_num = []
jPrecedent_link_get = []
jPrecedent_link_repl = []
court_name = []
court_num = []
court_detail = []
court_reference = []
court_Decision = []
page=1
url = f'{base_url}&page={page}&JO={JO}'
#&org={org}
response = requests.get(url)
title_XML = response.text
title_soup = BeautifulSoup(title_XML, 'xml') 

total_page = int(title_soup.select('totalCnt')[0].string)//100+1

분류 코드 
for page in tqdm(range(1, total_page+1)) :
    url = f'{base_url}&page={page}&org={org}&JO={JO}'
    response = requests.get(url)
    title_XML = response.text
    title_soup = BeautifulSoup(title_XML, 'xml') 

    for i in range(len(title_soup.select('사건명'))) :
        event_Title.append(title_soup.select('사건명')[i].string)
        event_num.append(title_soup.select('사건번호')[i].string)
        event_type.append(title_soup.select('사건종류명')[i].string)
        jPrecedent_num.append(title_soup.select('판례일련번호')[i].string)
        jPrecedent_link_get.append(title_soup.select('판례상세링크')[i].string)
        court_name.append(title_soup.select('법원명')[i].string)
        

for i in tqdm(range(len(jPrecedent_link_get))) :
    jPrecedent_link_repl.append(jPrecedent_link_get[i][:-10].replace('HTML', 'XML'))
    event_url = f'{event_base_url}{jPrecedent_link_repl[i]}'
    event_response = requests.get(event_url)
    main_XMLevent = event_response.text
    main_soup = BeautifulSoup(main_XMLevent, 'xml')
    #print(event_url)
    court_num.append(main_soup.select('사건번호')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_detail.append(main_soup.select('판례내용')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_reference.append(main_soup.select('참조조문')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_Decision.append(main_soup.select('판시사항')[0].string.replace('</br>', '').replace('<br/>', ''))


In [ ]:
import pandas as pd
'''
event_Title 
event_num 
event_type 
jPrecedent_num 
jPrecedent_link_get 
court_name 
court_num 
court_detail 
court_reference 
court_Decision
'''

df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, 'court_num':court_num, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_csv('court_work.csv',encoding='cp949')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import pandas as pd
import pickle

df = pd.read_csv('court_work.csv', encoding='cp949')
# 줄거리 텍스트 데이터 전처리
okt = Okt()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = [' '.join(okt.morphs(sentence)) for sentence in text]

# TF-IDF 벡터라이저 초기화 및 학습
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_text)

# 모델과 TF-IDF 매트릭스 저장
with open('court_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('court_tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# 전처리된 텍스트 저장
with open('court_processed_text.pkl', 'wb') as f:
    pickle.dump(processed_text, f)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import pandas as pd
import pickle

# 영화 데이터 로드
df = pd.read_csv('court_work.csv', encoding='cp949')

# 저장된 모델과 매트릭스 불러오기
with open('court_tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('court_tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

with open('court_processed_text.pkl', 'rb') as f:
    processed_text = pickle.load(f)

def recommend_court(input_sentence, top_n=5):
    # 입력 문장을 Okt를 사용하여 전처리
    okt = Okt()
    processed_input = ' '.join(okt.morphs(input_sentence))
    
    # 입력 문장을 TF-IDF 변환
    input_tfidf = tfidf_vectorizer.transform([processed_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    
    # 유사도 순으로 정렬된 상위 n개 인덱스 추출
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 유사도 0인 경우 처리
    recommendations = []
    for idx in top_indices:
        if cosine_similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], \
                                df.iloc[idx]['event_num'], \
                                df.iloc[idx]['event_type'], \
                                df.iloc[idx]['jPrecedent_num'], \
                                df.iloc[idx]['court_name'], \
                                df.iloc[idx]['court_num'], \
                                df.iloc[idx]['court_detail'], \
                                df.iloc[idx]['court_reference'], \
                                df.iloc[idx]['court_Decision'], \
                                cosine_similarities[idx]))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations

'''
event_Title 
event_num 
event_type 
jPrecedent_num 
court_name 
court_num 
court_detail 
court_reference 
court_Decision
'''

print('상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :')
court_input = str(input())
print('판례 및 판시 사항을 보시겠습니까?(Y/N) : ')
court_detail_OX = str(input())
input_sentence = court_input

# 판례
recommended_court = recommend_court(input_sentence)
if isinstance(recommended_court, str):
    print(recommended_court)
else:
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_num, court_detail, court_reference, court_Decision, similarity  in recommended_court:
        if (court_detail_OX == 'Y') or (court_detail_OX == 'y') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f})\n사건 번호 : {event_num} \n사건 종류 : {event_type} \n판례 일련번호 : {jPrecedent_num} \
            \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n 판례 내용 :\n{court_detail} \n\n 참조조문 \n{court_reference} \
            \n\n판시사항 :\n{court_Decision}\n\n\n")
        elif (court_detail_OX == 'N') or (court_detail_OX == 'n') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f})\n사건 번호 : {event_num} \n사건 종류 : {event_type} \n판례 일련번호 : {jPrecedent_num} \
            \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n 참조조문 \n{court_reference}\n\n\n")


In [ ]:
court_reference